## BETTER WAY 20 - 동적 기본 인수를 지정하려면 None과 docstring을 사용하자
## Item20 - Use None and Docstrings to Specify Dynamic Default Arguments

키워드 인수의 기본값(default)으로 비정적(non-static) 타입을 사용해야 할 때가 있다. <br />
예를 들어 이벤트 발생 시각까지 포함해 로깅 메시지를 출력한다고 하면, 아래와 같이 작성할 수 있다.

In [1]:
from datetime import datetime
from time import sleep

def log(message, when=datetime.now()):
    print('{}: {}'.format(when, message))
    
    
log('Hi there!')
sleep(1)
log('Hi again!')

2018-01-23 10:05:02.364772: Hi there!
2018-01-23 10:05:02.364772: Hi again!


위의 출력결과를 보면 `Hi there!`와 `Hi again!`의 출력 타임스탬프가 동일하다는 것을 알 수 있다. 그 이유는 `datetime.now()`메소드가 `log` 함수를 정의할 때 딱 한번만 실행되기 때문이다. <br />
이러한 문제가 발생하지 않게 하려면 기본값을 `None`으로 설정하고 *docstring(문서화 문자열)* 으로 실제 동작을 문서화 하는 것이 좋다. 아래의 코드와 같이 작성해 줄 수 있다.

In [2]:
def log(message, when=None):
    # docstring 작성
    """Log a message with a timestamp.
    
    Args:
        message: Message to print.
        when: datetime of when the message occurred.
            Defaults to the present time.
    """
    when = datetime.now() if when is None else when
    print('{}: {}'.format(when, message))
    

log('Hi there!')
sleep(1)
log('Hi again!')

2018-01-23 10:05:04.252790: Hi there!
2018-01-23 10:05:05.253453: Hi again!


기본인수(default) 값으로 `None`을 사용하는 방법은 인수가 수정 가능(mutuable)할 때 중요하다. 예를 들어 JSON 데이터로 인코딩된 값을 로드한다고 할 때, 데이터 디코딩이 실패하면 기본값(default)으로 빈 딕셔너리를 반환하는 코드는 아래와 같이 작성할 수 있다.

In [3]:
import json

def decode(data, default={}):
    try:
        return json.loads(data)
    except ValueError as e:
        return default

In [4]:
foo = decode('bad data')
foo['stuff'] = 5
bar= decode('also bad')
bar['meep'] = 1
print('Foo:',foo)
print('Bar:',bar)

Foo: {'meep': 1, 'stuff': 5}
Bar: {'meep': 1, 'stuff': 5}


위의 `decode`함수의 문제는 `foo`와 `bar`가 각각의 딕셔너리 `foo = {'stuff': 5}, bar = {'meep': 1}` 의 형태로 출력이 되어야 하는데 `foo`와 `bar`가 동일한 딕셔너리로 출력된다는 것이다. 그 이유는 기본값은 딱 한번만 특정되므로 모든 `decode`함수가 호출 될 때 마다 기본값인 빈 딕셔너리가 공유되기 때문이다. 따라서 `foo`와 `bar`가 같은 것이다.

In [5]:
assert foo is bar

키워드 인수의 기본값을 `None`으로 설정하고 함수의 `docstring`에 문서화하여 이 문제를 해결할 수 있다.

In [6]:
def decode(data, default=None):
    """Load JSON data from string.
    
    Agrs:
        data: JSON data to decode.
        default: Value to return if decoding fails.
            Defaults to an empty dictionary.
    """
    if default is None:
        default = {}
    try:
        return json.loads(data)
    except ValueError:
        return default

In [7]:
foo = decode('bad data')
foo['stuff'] = 5
bar= decode('also bad')
bar['meep'] = 1
print('Foo:',foo)
print('Bar:',bar)

Foo: {'stuff': 5}
Bar: {'meep': 1}


### 정리
- 기본 인수(default)는 모듈 로드 시점에 함수 정의 과정에서 딱 한번만 특정된다. 그렇기 때문에 ({} 나 []와 같은) 동적인 값에는 제대로 동작하지 않기도 한다.
- 값이 동적인 키워드 인수에는 기본값으로 `None`을 사용하는 것이 좋다. 그리고 함수의 docstring에 실제 기본 동작을 문서화하는 것이 좋다.